In [1]:
import pandas as pd

In [2]:
# Load and preprocess the general dataset

df = pd.read_excel(
    '../datasets/appliedai-use-case-library---risk-class-tum.ai.xlsx', header=2, engine='openpyxl')
df = df[:124]
df = df.drop(95, axis=0)
df = df[df.columns[[8, 20, 21, 22, 23, 24, 25]]]
df = df[df['AI System'] != 'No publicly described implementation is known at December 2021. The content of the database is regularly renewed.']
df

FileNotFoundError: [Errno 2] No such file or directory: 'appliedai-use-case-library---risk-class-tum.ai.xlsx'

In [ ]:
# Preprocess the dataset for risk classification

risk_classification_dataset = df[df.columns[[0, 1, 4]]]
risk_classification_dataset.loc[risk_classification_dataset['Is your AI System prohibited according to the AI Act?']
                                == 'Yes', 'Is the AI System high-risk or low risk?'] = 'prohibited'
risk_classification_dataset = risk_classification_dataset.drop(columns=[
    'Is your AI System prohibited according to the AI Act?'])


risk_classification_dataset

In [ ]:
risk_classification_dataset = risk_classification_dataset.join(pd.get_dummies(
    risk_classification_dataset['Is the AI System high-risk or low risk?']))
risk_classification_dataset['Risk'] = risk_classification_dataset[[
    'It is unclear', 'high-risk', 'low-risk', 'prohibited']].values.tolist()
risk_classification_dataset = risk_classification_dataset[[
    'AI System', 'Risk']]

risk_classification_dataset

In [ ]:
# Preprocess the dataset for explanations of prohibited cases

prohibited_explanations_dataset = df[df['Is your AI System prohibited according to the AI Act?'] == 'Yes'][[
    'AI System', 'Which of the items leads to this assessment?.1', 'If it is unclear, please explain why. Otherwise, please give a rational for choosing yes or no..1']].fillna('')
# -------------------------------------------------------------------------------
prohibited_explanations_dataset['explanation'] = prohibited_explanations_dataset['Which of the items leads to this assessment?.1'] + \
    prohibited_explanations_dataset['If it is unclear, please explain why. Otherwise, please give a rational for choosing yes or no..1']
prohibited_explanations_dataset = prohibited_explanations_dataset.drop(
    ['Which of the items leads to this assessment?.1', 'If it is unclear, please explain why. Otherwise, please give a rational for choosing yes or no..1'], axis=1)


prohibited_explanations_dataset

In [ ]:
# Preprocess the dataset for explanations of high-risk cases

high_risk_explanations_dataset = df[(df['Is the AI System high-risk or low risk?'] == 'high-risk') & (
    df['Is your AI System prohibited according to the AI Act?'] == 'No')][df.columns[[0, 6]]]
high_risk_explanations_dataset = high_risk_explanations_dataset.join(
    pd.get_dummies(high_risk_explanations_dataset.iloc[:, 1]))
high_risk_explanations_dataset['annex'] = high_risk_explanations_dataset[[
    1, 4, 6, 8]].values.tolist()
high_risk_explanations_dataset = high_risk_explanations_dataset[[
    'AI System', 'annex']]

# ECODING IS IN THE ORDER 1, 4, 6, 8

high_risk_explanations_dataset